In [14]:
import hashlib
import time
import json

# Block class representing each student data block
class Block:
    def __init__(self, index, student_data, previous_hash):
        self.index = index  # Block index
        self.timestamp = time.time()  # Time of block creation
        self.student_data = student_data  # Student information stored in the block
        self.previous_hash = previous_hash  # Hash of the previous block
        self.nonce = 0  # Used for proof-of-work (optional)
        self.hash = self.calculate_hash()  # Generate block hash

    def calculate_hash(self):
        """
        Generate SHA-256 hash for the block.
        """
        data_string = json.dumps(self.student_data, sort_keys=True) + str(self.timestamp) + self.previous_hash + str(self.nonce)
        return hashlib.sha256(data_string.encode()).hexdigest()

    def mine_block(self, difficulty):
        """
        Proof-of-work mining: Finds a hash that starts with 'difficulty' number of zeros.
        """
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()

# Blockchain class
class Blockchain:
    def __init__(self, difficulty=2):
        self.chain = []  # List to store the blockchain
        self.difficulty = difficulty  # Difficulty level for mining
        self.create_genesis_block()

    def create_genesis_block(self):
        """
        Creates the first block (Genesis Block) of the blockchain.
        """
        genesis_block = Block(
            index=0,
            student_data={"student_id": "0", "name": "Genesis Block", "department": "N/A"},
            previous_hash="0"
        )
        genesis_block.mine_block(self.difficulty)  # Mine the genesis block
        self.chain.append(genesis_block)

    def add_block(self, student_data):
        """
        Adds a new block with student information to the blockchain.
        """
        last_block = self.chain[-1]
        new_block = Block(index=len(self.chain), student_data=student_data, previous_hash=last_block.hash)
        new_block.mine_block(self.difficulty)  # Mine the new block
        self.chain.append(new_block)

    def delete_student(self, student_id):
        """
        Deletes a student block by ID by creating a new valid chain.
        """
        new_chain = [self.chain[0]]  # Start with genesis block
        for block in self.chain[1:]:
            if block.student_data["student_id"] != student_id:
                new_chain.append(block)
        self.chain = new_chain

    def is_chain_valid(self):
        """
        Checks if the blockchain is valid (prevents data tampering).
        """
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            # Check if current block hash is valid
            if current_block.hash != current_block.calculate_hash():
                print(f"Block {current_block.index} has been tampered with!")
                return False

            # Check if the previous hash matches the stored previous hash
            if current_block.previous_hash != previous_block.hash:
                print(f"Block {current_block.index} has an invalid previous hash!")
                return False

        print("Blockchain is valid!")
        return True

    def print_chain(self):
        """
        Prints the entire blockchain.
        """
        for block in self.chain:
            print(f"Block Index: {block.index}")
            print(f"Block Hash: {block.hash}")
            print(f"Student Data: {block.student_data}")
            print(f"Timestamp: {block.timestamp}")
            print(f"Previous Hash: {block.previous_hash}")
            print(f"Nonce: {block.nonce}")
            print("-" * 80)

# Example usage
if __name__ == "__main__":
    blockchain = Blockchain(difficulty=3)  # Higher difficulty increases security

    # Add student information
    blockchain.add_block({"student_id": "1", "name": "Md Saniul basir Saz", "department": "CSE"})
    blockchain.add_block({"student_id": "2", "name": "John Doe", "department": "EEE"})
    blockchain.add_block({"student_id": "3", "name": "Jane Smith", "department": "BBA"})

    # Print the blockchain before deletion
    print("Blockchain before deletion:")
    blockchain.print_chain()


    blockchain.delete_student("5")

    # Print the blockchain after deletion
    print("Blockchain after deletion:")
    blockchain.print_chain()

    # Validate the blockchain
    blockchain.is_chain_valid()


Blockchain before deletion:
Block Index: 0
Block Hash: 000c4c56729bad0db2dbc74863fc47de89680ec85c3d296b68694a1c8d3fa21d
Student Data: {'student_id': '0', 'name': 'Genesis Block', 'department': 'N/A'}
Timestamp: 1738839652.3690677
Previous Hash: 0
Nonce: 3396
--------------------------------------------------------------------------------
Block Index: 1
Block Hash: 000c8a511b6e4ee6bf5878bb8da96e36a4f76f29457d6a525e25f36281ddb4b9
Student Data: {'student_id': '1', 'name': 'Md Saniul basir Saz', 'department': 'CSE'}
Timestamp: 1738839652.385605
Previous Hash: 000c4c56729bad0db2dbc74863fc47de89680ec85c3d296b68694a1c8d3fa21d
Nonce: 3164
--------------------------------------------------------------------------------
Block Index: 2
Block Hash: 0001cb40fdf00d001179c66cc1754fb192e6c1551c260f1fe42f4f4d20c6988f
Student Data: {'student_id': '2', 'name': 'John Doe', 'department': 'EEE'}
Timestamp: 1738839652.4078753
Previous Hash: 000c8a511b6e4ee6bf5878bb8da96e36a4f76f29457d6a525e25f36281ddb4b9
Non

In [1]:
import hashlib
import time
import json
import pandas as pd

# Block class
class Block:
    def __init__(self, student_data, previous_hash):
        self.timestamp = time.time()
        self.student_data = student_data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        data_string = json.dumps(self.student_data, sort_keys=True) + str(self.timestamp) + self.previous_hash
        return hashlib.sha256(data_string.encode()).hexdigest()

# Blockchain class
class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(
            student_data={"student_id": "0", "name": "Genesis", "department": "N/A"},
            previous_hash="0"
        )
        self.chain.append(genesis_block)

    def add_block(self, student_data):
        last_block = self.chain[-1]
        new_block = Block(student_data, last_block.hash)
        self.chain.append(new_block)

    def soft_delete_student(self, student_id):
        for block in self.chain:
            if block.student_data["student_id"] == student_id:
                block.student_data["deleted"] = True  # Soft delete
                print(f"Student {student_id} marked as deleted.")
                return
        print(f"Student {student_id} not found in the blockchain.")

    def print_chain(self):
        for block in self.chain:
            print(f"Block Hash: {block.hash}")
            print(f"Student Data: {block.student_data}")
            print(f"Timestamp: {block.timestamp}")
            print(f"Previous Hash: {block.previous_hash}")
            print("-" * 50)

# Load student data from JSON file
def load_students_from_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Generate dummy student data and save to JSON file
def generate_dummy_students(file_path, num_students=1000):
    students = []
    for i in range(1, num_students + 1):
        student = {
            "student_id": str(i),
            "name": f"Student {i}",
            "department": "CSE",
            "deleted": False
        }
        students.append(student)
    with open(file_path, 'w') as file:
        json.dump(students, file, indent=4)

# Example usage
students_file = "students.json"
generate_dummy_students(students_file, num_students=1000)  # Generate 1000 dummy data

blockchain = Blockchain()

# Load students from JSON file
students = load_students_from_json(students_file)

# Add students to blockchain
for student in students:
    blockchain.add_block(student)

# Print blockchain
blockchain.print_chain()

# Delete a student (soft delete)
blockchain.soft_delete_student("4")  # Example student ID


Block Hash: d6bac66d7c71358ac2fb4d7165d06776e2698d255efd590ecd5d09c0284bff14
Student Data: {'student_id': '0', 'name': 'Genesis', 'department': 'N/A'}
Timestamp: 1738842447.3882267
Previous Hash: 0
--------------------------------------------------
Block Hash: 2d4cbc9370e5c1ae787fd0573884670114d55c78bb2a86c5d77f80c5993080f3
Student Data: {'student_id': '1', 'name': 'Student 1', 'department': 'CSE', 'deleted': False}
Timestamp: 1738842447.3882267
Previous Hash: d6bac66d7c71358ac2fb4d7165d06776e2698d255efd590ecd5d09c0284bff14
--------------------------------------------------
Block Hash: 04977be556919387a17c20b4fb14ab7add94e76ec22eeb69c6d8bb65de206f9a
Student Data: {'student_id': '2', 'name': 'Student 2', 'department': 'CSE', 'deleted': False}
Timestamp: 1738842447.3882267
Previous Hash: 2d4cbc9370e5c1ae787fd0573884670114d55c78bb2a86c5d77f80c5993080f3
--------------------------------------------------
Block Hash: 373b29e7840c12a60dfb746368826e73b33b5748b6f6a63b3a1720942e782785
Student Da